In [162]:
!pip install texthero
import pandas as pd
import texthero as hero
from texthero import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import HashingVectorizer
from imblearn.over_sampling import SMOTE
import numpy as np

In [163]:
#cols = ['EventDescription','FailedAssets','IncidentCause','IncidentConsequence','IncidentType','WeatherStation','Status','Category']
cols = ['EventDescription','IncidentCause','IncidentConsequence','Category']

In [164]:
df = pd.read_csv('/content/drive/MyDrive/ESV Data/cleaned_incidents1.csv', usecols= cols)
df = df.dropna()
df

,EventDescription,IncidentCause,IncidentConsequence,Category
0,A nearby customer reported sparking of electri...,HVABC cable faulted midspan,Grassfire,OH Cable
1,A contractor reported that he had contacted an...,Contractor contacted earthing conductor,No Go Zone (contact),Dug up
2,A field crew attending an outage found that a ...,Conductor broke due to rust,Serious risk to public safety (e.g. live condu...,Connection
3,Interfere and vandalism in substation. Unknown...,Vandalism,Loss of supply;Damage to network assets,Other
4,A nearby customer reported that a high load ha...,Unknown high load contacted LV service cable,No Go Zone (contact);Damage to network assets,Vehicle
...,...,...,...,...
6499,A report came in to UE of a FMB sparking at 31...,The likely cause was determined by EIO to be a...,Plant/equipment,Connection
6500,Report received from a resident to advise that...,Crane struck line,No Go Zone (contact);Damage to network assets,Vehicle
6501,Report received of pole fire at incident locat...,Excessive leakage current,Pole-top fire,Pole
6502,A customer called to report a tractor had hit ...,Farm equipment hit pole,Damage to network assets,Vehicle


In [165]:
df['Description'] = df['EventDescription'] + ' '+ df['IncidentCause']+ ' ' + df['IncidentConsequence']

In [166]:
df['Description']

0       A nearby customer reported sparking of electri...
1       A contractor reported that he had contacted an...
2       A field crew attending an outage found that a ...
3       Interfere and vandalism in substation. Unknown...
4       A nearby customer reported that a high load ha...
                              ...                        
6499    A report came in to UE of a FMB sparking at 31...
6500    Report received from a resident to advise that...
6501    Report received of pole fire at incident locat...
6502    A customer called to report a tractor had hit ...
6503    concrete electrical cover outside of front of ...
Name: Description, Length: 6488, dtype: object

In [167]:
df['clean_description'] = hero.clean(df['Description'])
df['clean_description']

0       nearby customer reported sparking electrical l...
1       contractor reported contacted earthing cable e...
2       field crew attending outage found 22kv conduct...
3       interfere vandalism substation unknown third p...
4       nearby customer reported high load pulled wire...
                              ...                        
6499    report came ue fmb sparking st kilda st bright...
6500    report received resident advise crane made con...
6501    report received pole fire incident location ar...
6502    customer called report tractor hit wire brough...
6503    concrete electrical cover outside front house ...
Name: clean_description, Length: 6488, dtype: object

In [168]:
le = LabelEncoder()
df['Category'] = le.fit_transform(df['Category'])
df['Category']

0        9
1        5
2        3
3       10
4       14
        ..
6499     3
6500    14
6501    11
6502    14
6503     0
Name: Category, Length: 6488, dtype: int64

In [169]:
from texthero import preprocessing
#create a custom cleaning pipeline
custom_pipeline = [preprocessing.fillna
                   , preprocessing.lowercase
                   , preprocessing.remove_digits
                   , preprocessing.remove_punctuation
                   , preprocessing.remove_diacritics
                   #, preprocessing.remove_stopwords
                   , preprocessing.remove_whitespace
                   , preprocessing.stem]
#pass the custom_pipeline to the pipeline argument
df['clean_description'] = hero.clean(df['Description'], pipeline = custom_pipeline)
df.head()

,EventDescription,IncidentCause,IncidentConsequence,Category,Description,clean_description
0,A nearby customer reported sparking of electri...,HVABC cable faulted midspan,Grassfire,9,A nearby customer reported sparking of electri...,a nearbi custom report spark of electr line at...
1,A contractor reported that he had contacted an...,Contractor contacted earthing conductor,No Go Zone (contact),5,A contractor reported that he had contacted an...,a contractor report that he had contact an ear...
2,A field crew attending an outage found that a ...,Conductor broke due to rust,Serious risk to public safety (e.g. live condu...,3,A field crew attending an outage found that a ...,a field crew attend an outag found that a 22kv...
3,Interfere and vandalism in substation. Unknown...,Vandalism,Loss of supply;Damage to network assets,10,Interfere and vandalism in substation. Unknown...,interfer and vandal in substat unknown third p...
4,A nearby customer reported that a high load ha...,Unknown high load contacted LV service cable,No Go Zone (contact);Damage to network assets,14,A nearby customer reported that a high load ha...,a nearbi custom report that a high load had pu...


In [170]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
x_train, x_test, y_train, y_test = train_test_split(df['clean_description'], df['Category'], random_state=1, test_size=0.25)

In [171]:
print(x_train.shape,y_train.shape)
print(x_test.shape, y_test.shape)

(4866,) (4866,)
(1622,) (1622,)


In [172]:
'''
def tfidf():
    vectorizer = TfidfVectorizer(analyzer='word', max_features=1000)
    vectorizer.fit(df['clean_description'])
    train_tfidf = vectorizer.transform(x_train)
    test_tfidf = vectorizer.transform(x_test)
    return train_tfidf, test_tfidf

# bow
def bow():
    count_vectorizer = CountVectorizer(analyzer='word', max_features=1000)
    count_vectorizer.fit(df['clean_description'])
    train_bow = count_vectorizer.fit_transform(x_train)
    test_bow = count_vectorizer.fit_transform(x_test)
    return train_bow, test_bow

# bigrams
def bigrams():
    bigram_vectorizer = CountVectorizer(analyzer='word', ngram_range=(2,2), max_features=1000)
    bigram_vectorizer.fit(df['clean_description'])
    train_bigram = bigram_vectorizer.fit_transform(x_train)
    test_bigram = bigram_vectorizer.fit_transform(x_test)
    return train_bigram, test_bigram

'''

"\ndef tfidf():\n    vectorizer = TfidfVectorizer(analyzer='word', max_features=1000)\n    vectorizer.fit(df['clean_description'])\n    train_tfidf = vectorizer.transform(x_train)\n    test_tfidf = vectorizer.transform(x_test)\n    return train_tfidf, test_tfidf\n\n# bow\ndef bow():\n    count_vectorizer = CountVectorizer(analyzer='word', max_features=1000)\n    count_vectorizer.fit(df['clean_description'])\n    train_bow = count_vectorizer.fit_transform(x_train)\n    test_bow = count_vectorizer.fit_transform(x_test)\n    return train_bow, test_bow\n\n# bigrams\ndef bigrams():\n    bigram_vectorizer = CountVectorizer(analyzer='word', ngram_range=(2,2), max_features=1000)\n    bigram_vectorizer.fit(df['clean_description'])\n    train_bigram = bigram_vectorizer.fit_transform(x_train)\n    test_bigram = bigram_vectorizer.fit_transform(x_test)\n    return train_bigram, test_bigram\n\n"

Feature Extraction Technique - TFIDF

In [173]:
vectorizer = TfidfVectorizer(analyzer='word', max_features=5000)
vectorizer.fit(df['clean_description'])
train_tfidf = vectorizer.transform(x_train)
test_tfidf = vectorizer.transform(x_test)


print(train_tfidf.shape, test_tfidf.shape)

(4866, 5000) (1622, 5000)


In [174]:
svc = SVC(C=1.0, kernel = 'linear')
svc.fit(train_tfidf, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [175]:
accuracy = accuracy_score(y_test,svc.predict(test_tfidf))

In [176]:
print(accuracy)

0.8372379778051788


Count_vectorizer 

In [177]:
count_vectorizer = CountVectorizer(ngram_range=(1,2))
count_vectorizer.fit(df['clean_description'])
train_bow = count_vectorizer.transform(x_train)
test_bow = count_vectorizer.transform(x_test)

In [178]:
print(y_test.shape, test_bow.shape)
print(y_train.shape, train_bow.shape)

(1622,) (1622, 98419)
(4866,) (4866, 98419)


In [179]:
svc = SVC(C=1.0, kernel = 'linear')
svc.fit(train_bow, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [180]:
print(accuracy_score(y_test,svc.predict(test_bow)))

0.8236744759556104


Hashing Vectorizer

In [186]:
hv = HashingVectorizer(n_features=5000)

In [187]:
hv.fit(df['clean_description'])
train_hv = hv.transform(x_train)
test_hv = hv.transform(x_test)

In [188]:
svc.fit(train_hv, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [189]:
accuracy = accuracy_score(y_test,svc.predict(test_hv))

In [190]:
print(accuracy)

0.812577065351418


SMOTE upsampling technique

In [191]:
oversample = SMOTE(random_state=0,n_jobs=-1,k_neighbors=5)
train_tfidf, y_train = oversample.fit_resample(train_tfidf, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.



In [194]:
y_train

array([14, 11,  3, ..., 14, 14, 14])

In [197]:
svc.fit(train_tfidf, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [198]:
print(accuracy_score(y_test,svc.predict(test_tfidf)))

0.8304562268803946


Findings:
1. Trying out only 3 features 'EventDescription', 'IncidentCause', 'IncidentConsequence'. 
2. SMOTE + TFIDF(analyzer='word', max_features=5000) gives 83% accuracy
3. Countvectorizer with n_grams(1,2) alone gives 82% accuracy
4. TFIDF(analyzer='word', max_features=5000) gives 83% accuracy
5. Only hashvectoorizer(n_features=5000) it gave 81.2% accuracy